# Setup

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import copy
import functools
import itertools
import logging
import re
import time
import yaml
from collections import Counter

import humanize
from pymicro.file import file_utils
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import scipy as sp
from numpy import ndarray
from matplotlib import pyplot as plt, patches, cm
from sklearn import metrics as met
from progressbar import progressbar as pbar
from tensorflow.keras import layers
from sklearn import metrics, preprocessing
from sklearn import model_selection

import tomo2seg.data as tomo2seg_data
from tomo2seg.data import ModelPaths, Volume, EstimationVolume, SetPartition
from tomo2seg.volume_img_segm import VolumeImgSegmSequence
from tomo2seg import viz
from tomo2seg.logger import logger

In [4]:
logger.setLevel(logging.DEBUG)

In [4]:
random_state = 42
random_state = np.random.RandomState(random_state)

# Options

In [5]:
analyze_crops = False

# Load from the disk

In [13]:
# Metadata/paths objects

## Model
model_paths = ModelPaths(model_name := "dryrun-03")
logger.info(f"{model_paths=}")

## Volume
volume = Volume.with_check(
    volume_name := tomo2seg_data.VOLUME_PRECIPITATES_DRYRUN, 
    volume_version := "val"  # this is not supposed to be used like this...
)
logger.info(f"{volume=}")

hst_read = lambda x: functools.partial(
    # from pymicro
    file_utils.HST_read,
    
    # pre-loaded kwargs
    autoparse_filename=False,  # the file names are not properly formatted
    data_type=volume.metadata.dtype,
    dims=volume.metadata.dimensions,
    verbose=True,
)(str(x))  # it doesn't accept paths...

logger.info("Loading data from disk.")

## Data
data_volume = hst_read(volume.data_path)
logger.debug(f"{data_volume.shape=}")

## Labels
labels_volume = hst_read(volume.labels_path)
logger.debug(f"{labels_volume.shape=}")